In [384]:
import sys
import os
repo_path = os.path.abspath('../')
sys.path.append(repo_path)
import pandas as pd
import geopandas as gpd
import geoplot as gplt
import scripts.addSA2 as addSA2
import numpy as np

In [385]:
df = pd.read_csv(f"../data/curated/pre_processed_data.csv")
df = addSA2.addSA2(df, use_postcode = False)
df = df.loc[df["SA2"] != 0]

In [386]:
df

,url,postcode,suburb,state,weekly_rent,bond,num_beds,num_baths,num_parking,property_type,...,park_duration,park_distance,shop_duration,shop_distance,train_duration,train_distance,stop_duration,stop_distance,SA2,geometry
0,https://www.domain.com.au/40-esmond-street-ard...,3022,esmond,vic,400.0,1738.0,3.0,2.0,1.0,House,...,0.101667,42.6,5.595000,2903.6,8.285000,5710.8,1.300000,639.8,213011328.0,"POLYGON ((144.79438 -37.77834, 144.79467 -37.7..."
1,https://www.domain.com.au/11-grant-avenue-gisb...,3437,gisborne,vic,600.0,2608.0,4.0,2.0,1.0,House,...,16.075000,15921.0,17.856667,17363.3,8.270000,6053.8,1.345000,489.0,210021232.0,"POLYGON ((144.57914 -37.45977, 144.57978 -37.4..."
2,https://www.domain.com.au/15-balmoral-avenue-b...,3083,balmoral,vic,575.0,2499.0,3.0,1.0,3.0,House,...,1.670000,423.2,7.698333,3781.9,5.788333,2812.4,1.083333,298.9,209011196.0,"POLYGON ((145.05892 -37.69784, 145.05918 -37.6..."
3,https://www.domain.com.au/6-39-wellington-stre...,3182,st kilda,vic,600.0,2607.0,2.0,3.0,2.0,Apartment / Unit / Flat,...,1.280000,518.2,3.696667,1723.3,2.758333,962.0,1.946667,847.3,206051513.0,"POLYGON ((144.98225 -37.86154, 144.98226 -37.8..."
4,https://www.domain.com.au/7-9-sheffield-street...,3058,coburg,vic,370.0,NaN,2.0,1.0,1.0,Apartment / Unit / Flat,...,0.783333,404.1,0.958333,453.9,1.531667,711.8,0.508333,266.6,206011497.0,"POLYGON ((144.96328 -37.74286, 144.96351 -37.7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14485,https://www.domain.com.au/86-centenary-street-...,3198,seaford,vic,480.0,2086.0,3.0,1.0,4.0,House,...,10.938333,11765.7,NaN,NaN,8.471667,2910.4,1.858333,582.3,214011375.0,"POLYGON ((145.12445 -38.10103, 145.12447 -38.1..."
14486,https://www.domain.com.au/7-wingara-avenue-kei...,3033,keilor,vic,420.0,1825.0,2.0,1.0,1.0,House,...,0.345000,120.6,NaN,NaN,10.538333,6342.8,1.035000,481.8,210011534.0,"POLYGON ((144.85471 -37.73808, 144.85596 -37.7..."
14487,https://www.domain.com.au/3-32-loller-street-b...,3186,brighton,vic,560.0,2433.0,2.0,1.0,1.0,Apartment / Unit / Flat,...,0.790000,471.7,NaN,NaN,1.368333,556.1,0.698333,374.1,208011169.0,"POLYGON ((144.98392 -37.90034, 144.98387 -37.9..."
14488,https://www.domain.com.au/42-george-street-oak...,3166,oakleigh,vic,490.0,2129.0,3.0,1.0,1.0,House,...,0.563333,239.8,2.566667,1414.5,4.000000,2574.9,0.311667,227.1,212051326.0,"POLYGON ((145.11480 -37.89791, 145.11526 -37.8..."


In [387]:
sa2 = pd.read_csv(f"../data/raw/abs_data/pop_data.csv")
sa2 = sa2[sa2["TIME_PERIOD: Time Period"] == 2021][["ASGS_2021: Region", "OBS_VALUE"]]
sa2["ASGS_2021: Region"] = sa2["ASGS_2021: Region"].apply(lambda x: x.split(':')[0])
sa2 = sa2.rename(columns = {"ASGS_2021: Region":"SA2", "OBS_VALUE":"SA2_population"})
sa2 = sa2.astype(float)
building_metrics_2019 = ["sa2_code16", "tot_dwl_num", "value_non_resial_building_aud000"]
buildings_2019 = pd.read_csv(f"../data/raw/datasource-AU_Govt_ABS-UoM_AURIN_DB_3 abs_building_approvals_sa2_2018_19.csv")[building_metrics_2019]
buildings_2019 = buildings_2019.rename(columns = {"tot_dwl_num":"new_dwellings_2019", "value_non_resial_building_aud000":"non_residential_value_2019"})

building_metrics_2021 = ["sa2_code", "total_dwellings_no", "value_of_non_residential_building_000"]
buildings_2021 = pd.read_csv(f"../data/raw/datasource-AU_Govt_ABS-UoM_AURIN_DB_3 abs_building_approvals_sa2_2020_21.csv")[building_metrics_2021]
buildings_2021 = buildings_2021.rename(columns = {"total_dwellings_no":"new_dwellings_2021", "value_of_non_residential_building_000":"non_residential_value_2021"})

sa2 = sa2.join(buildings_2019.set_index("sa2_code16"), on="SA2")
sa2 = sa2.join(buildings_2021.set_index("sa2_code"), on="SA2")
sa2 = sa2.fillna(0)

In [388]:
df.loc[:,"count"] = 1
df = df[["suburb", "SA2", "count"]]

In [389]:
sub_count = df.groupby(["suburb","SA2"]).sum().reset_index()

In [390]:
sa2_count = df[["SA2", "count"]].groupby("SA2").sum()
sa2_count = sa2_count.rename(columns = {"count":"sa2_count"})

In [391]:
scores = sub_count.join(sa2_count, on = "SA2")
scores["SA2_fraction"] = scores["count"]/scores["sa2_count"]

In [392]:
scores = scores.join(sa2.set_index("SA2"), on = "SA2")

In [393]:
scores

,suburb,SA2,count,sa2_count,SA2_fraction,SA2_population,new_dwellings_2019,non_residential_value_2019,new_dwellings_2021,non_residential_value_2021
0,abbeyard,212031558.0,1,20,0.050000,12432.0,0.0,0.0000,538.0,685.2
1,abbotsford,206041506.0,4,117,0.034188,15998.0,0.0,0.0000,36.0,387793.4
2,abbotsford,206071139.0,46,49,0.938776,9262.0,4.0,7104.1001,3.0,67877.6
3,abeckett street,206041504.0,18,228,0.078947,17439.0,0.0,0.0000,0.0,289109.5
4,abeckett street,206061136.0,1,181,0.005525,19460.0,138.0,25646.1992,89.0,82187.8
...,...,...,...,...,...,...,...,...,...,...
2262,yarrawonga,216021414.0,11,13,0.846154,8585.0,95.0,33300.1992,144.0,1690.8
2263,yarrowee,201011484.0,1,24,0.041667,13062.0,0.0,0.0000,48.0,37737.7
2264,yea,204011062.0,1,2,0.500000,4142.0,33.0,1319.3000,46.0,1065.1
2265,yering,212011550.0,1,7,0.142857,7563.0,0.0,0.0000,215.0,4645.4


In [394]:
scores["suburb_population"] = scores["SA2_population"]*scores["SA2_fraction"]
scores["new_dwellings_2019"] = scores["new_dwellings_2019"]*scores["SA2_fraction"]
scores["non_residential_value_2019"] = scores["non_residential_value_2019"]*scores["SA2_fraction"]*1000 # convert to $s
scores["new_dwellings_2021"] = scores["new_dwellings_2021"]*scores["SA2_fraction"]
scores["non_residential_value_2021"] = scores["non_residential_value_2021"]*scores["SA2_fraction"]*1000 # convert to $s
suburb_pop = scores[["suburb","suburb_population", "new_dwellings_2019", "non_residential_value_2019",
                     "new_dwellings_2021", "non_residential_value_2021"]].groupby("suburb").sum().reset_index()

In [413]:
suburb_sa2 = scores[["suburb", "SA2"]].groupby("suburb").agg(lambda x: x.mode().to_list()[0])
suburb_sa2

,SA2
suburb,
abbeyard,212031558.0
abbotsford,206041506.0
abeckett street,206041504.0
aberfeldie,206031501.0
aberfeldy,209041437.0
...,...
yarrawonga,216021414.0
yarrowee,201011484.0
yea,204011062.0


In [396]:
h_df = pd.read_csv(f"../data/curated/historical_sales_clean.csv")

In [397]:
h_df = h_df[["suburb", "num_beds", "property_type", "2022_n_sold", "clearance", "avg_days_on_market", "2022_median", "2022_growth",
             "2021_growth", "2020_growth", "2019_growth", "2019_n_sold"]]
h_df = h_df.replace('-', 0).astype({"avg_days_on_market":"float"})
h_df

,suburb,num_beds,property_type,2022_n_sold,clearance,avg_days_on_market,2022_median,2022_growth,2021_growth,2020_growth,2019_growth,2019_n_sold
0,melbourne,2,house,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,melbourne,3,house,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,melbourne,1,unit,387.0,0.40,120.0,330000.0,0.043,0.006,0.006,0.006,479.0
3,melbourne,2,unit,596.0,0.41,147.0,540000.0,0.061,0.042,NaN,0.017,608.0
4,melbourne,3,unit,78.0,0.55,180.0,961000.0,NaN,0.009,0.057,0.010,117.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5150,inverloch,4,house,66.0,NaN,78.0,1158000.0,0.316,0.294,0.046,0.016,59.0
5151,inverloch,5,house,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
5152,inverloch,1,unit,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
5153,inverloch,2,unit,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [398]:
h_df.loc[:,'2022_growth'] = h_df['2022_growth'].fillna(h_df.groupby("suburb")['2022_growth'].transform("mean"))
h_df.loc[:,'2021_growth'] = h_df['2021_growth'].fillna(h_df.groupby("suburb")['2021_growth'].transform("mean"))
h_df.loc[:,'2020_growth'] = h_df['2020_growth'].fillna(h_df.groupby("suburb")['2020_growth'].transform("mean"))
h_df.loc[:,'2019_growth'] = h_df['2019_growth'].fillna(h_df.groupby("suburb")['2019_growth'].transform("mean"))
h_df.loc[:,'2019_n_sold'] = h_df['2019_n_sold'].fillna(0)
h_df.loc[:,'2022_n_sold'] = h_df['2022_n_sold'].fillna(0)
h_df.loc[:,'clearance'] = h_df['clearance'].fillna(h_df.groupby("suburb")['clearance'].transform("mean"))
h_df.loc[:,'avg_days_on_market'] = h_df['avg_days_on_market'].fillna(h_df.groupby("suburb")['avg_days_on_market'].transform("mean"))

In [399]:
h_df.loc[:,"total_growth"] = (1+h_df["2019_growth"])*(1+h_df["2020_growth"])*(1+h_df["2021_growth"]) - 1

In [400]:
h_df

,suburb,num_beds,property_type,2022_n_sold,clearance,avg_days_on_market,2022_median,2022_growth,2021_growth,2020_growth,2019_growth,2019_n_sold,total_growth
0,melbourne,2,house,0.0,0.4975,0.0,NaN,0.053200,0.043200,0.0424,0.034500,0.0,0.124948
1,melbourne,3,house,0.0,0.4975,0.0,NaN,0.053200,0.043200,0.0424,0.034500,0.0,0.124948
2,melbourne,1,unit,387.0,0.4000,120.0,330000.0,0.043000,0.006000,0.0060,0.006000,479.0,0.018108
3,melbourne,2,unit,596.0,0.4100,147.0,540000.0,0.061000,0.042000,0.0424,0.017000,608.0,0.104646
4,melbourne,3,unit,78.0,0.5500,180.0,961000.0,0.053200,0.009000,0.0570,0.010000,117.0,0.077178
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5150,inverloch,4,house,66.0,NaN,78.0,1158000.0,0.316000,0.294000,0.0460,0.016000,59.0,0.375180
5151,inverloch,5,house,0.0,NaN,0.0,NaN,0.313667,0.168667,0.0320,0.073667,0.0,0.294911
5152,inverloch,1,unit,0.0,NaN,0.0,NaN,0.313667,0.168667,0.0320,0.073667,0.0,0.294911
5153,inverloch,2,unit,0.0,NaN,0.0,NaN,0.313667,0.168667,0.0320,0.073667,0.0,0.294911


In [401]:
sums = pd.DataFrame(h_df.groupby("suburb").sum()["2019_n_sold"].reset_index())
sums = sums.rename(columns={"2019_n_sold":"total_sold"})
sums

,suburb,total_sold
0,abbotsford,195.0
1,aberfeldie,35.0
2,aintree,84.0
3,aireys-inlet,16.0
4,airport-west,146.0
...,...,...
1191,yarroweyah,0.0
1192,yellingbo,0.0
1193,yelta,0.0
1194,yinnar-south,0.0


In [402]:
h_df = h_df.join(sums.set_index("suburb"), on = "suburb")

In [403]:
h_df["total_growth"] = (h_df["2019_n_sold"]/h_df["total_sold"])*h_df["total_growth"]
h_df["2022_median"] = (h_df["2019_n_sold"]/h_df["total_sold"])*h_df["2022_median"]
h_df["2022_growth"] = (h_df["2019_n_sold"]/h_df["total_sold"])*h_df["2022_growth"]
h_df["clearance"] = (h_df["2019_n_sold"]/h_df["total_sold"])*h_df["clearance"]
h_df["avg_days_on_market"] = (h_df["2019_n_sold"]/h_df["total_sold"])*h_df["avg_days_on_market"]

In [404]:
fin_df = h_df[["suburb", "clearance", "avg_days_on_market", "total_growth", "2019_n_sold", "2022_n_sold", "2022_median", "2022_growth"]]
fin_df = fin_df.groupby("suburb").sum().reset_index()

In [405]:
fin_df = fin_df.rename(columns={"total_growth": "3_year_growth"})

In [406]:
fin_df = fin_df.join(suburb_pop.set_index("suburb"), on="suburb").fillna(0)

In [409]:
fin_df =

,suburb,clearance,avg_days_on_market,3_year_growth,2019_n_sold,2022_n_sold,2022_median,2022_growth,suburb_population,new_dwellings_2019,non_residential_value_2019,new_dwellings_2021,non_residential_value_2021
0,abbotsford,0.507077,71.774359,0.176514,195.0,294.0,7.657590e+05,0.049467,9241.878946,3.755102,6.669155e+06,4.047096,7.697972e+07
1,aberfeldie,0.637714,46.742857,0.399867,35.0,49.0,1.962143e+06,0.205971,2462.173913,0.000000,0.000000e+00,10.195652,1.585941e+06
2,aintree,0.400000,35.940476,0.235013,84.0,143.0,7.098214e+05,0.097869,256.025000,6.437500,2.231975e+05,4.337500,7.999375e+04
3,aireys-inlet,0.000000,35.000000,0.708938,16.0,22.0,1.935000e+06,0.423000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
4,airport-west,0.659863,59.397260,0.174302,146.0,186.0,8.415274e+05,0.067705,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191,yarroweyah,0.000000,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
1192,yellingbo,0.000000,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
1193,yelta,0.000000,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
1194,yinnar-south,0.000000,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00


In [410]:
fin_df.to_csv(f"../data/curated/future_prediction_data.csv")